In [48]:
import pandas as pd
import nltk.data
import numpy as np
from pandas import Series, DataFrame
import math
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import fnmatch
import re
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [49]:
cars = pd.read_csv("entry_lux_sedan.csv",header = None, encoding ='utf8')
comments = cars[1][:5000] #taking 5k comments for computational efficiency, exponentially faster than 10k
stop = stopwords.words('english')
def WordProcess(series):
    """This function will tokenize, remove stopwords and html tags in each row in a given series and return a list"""
    tokenized = []
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(series)):
        tokenized.append(tokenizer.tokenize(series[i]))
    tokenized_rmstop = []
    for i in range(len(tokenized)):
        lists = []
        for j in range(len(tokenized[i])):
            if tokenized[i][j] not in stop:
                lists.append(tokenized[i][j])
        tokenized_rmstop.append(lists)
    # remove the html words
    for i in range(len(tokenized_rmstop)):
        tokenized_rmstop[i] = tokenized_rmstop[i][3:] 
        tokenized_rmstop[i] = [element.lower() for element in tokenized_rmstop[i]]
    return tokenized_rmstop

In [50]:
comments_list = WordProcess(comments)

In [51]:
#model to brand mapping
models = pd.read_csv("models.csv")
car_brand = list(models['Replace'])
model_list = list(models['Search'])
#remove special characters
rm_special_characters = fnmatch.filter(model_list, '*\xa0')
new_mod_list = []
for eachmodel in model_list:
    if eachmodel in rm_special_characters:
        eachmodel = ''.join(eachmodel.split())[:-1].lower()
    if eachmodel not in rm_special_characters:
        eachmodel = eachmodel.lower()
    new_mod_list.append(eachmodel)
models['Search_new'] = Series(new_mod_list, index = models.index)
new_models = list(models['Search_new'])
models[:5]

,Replace,Search,Search_new
0,acura,integra,integra
1,acura,Legend�,legend
2,acura,vigor,vigor
3,acura,rlx,rlx
4,acura,ILX,ilx


In [52]:
#mapped each model to its respective car brand
for i in range(len(comments_list)):
    for (index, word) in enumerate(comments_list[i]):
        for (ind, eachmodel) in enumerate(new_models):
            if eachmodel == word:
                comments_list[i][index] = car_brand[ind]             

In [53]:
whole_thing = sum(comments_list,[])

In [54]:
car_brand_set = list(set(car_brand))
brand_counts = [(brand,whole_thing.count(brand)) for brand in car_brand_set]  
df_brand = pd.DataFrame(brand_counts)
df_brand = df_brand.rename(columns={0: 'Brand', 1: 'Frequency'}) 
df_brand = df_brand.sort_index(by='Frequency', ascending = False)
df_brand

,Brand,Frequency
22,car,5311
16,bmw,3082
21,acura,1953
2,audi,1665
15,infiniti,1069
11,honda,908
17,sedan,646
23,nissan,444
28,problem,413
24,toyota,405


In [55]:
df_brand.to_csv('top brands.csv', encoding ='utf-8')

In [167]:
#Calculate lift ratios
top_10 = ['bmw', 'acura', 'audi', 'infiniti', 'honda', 'nissan', 'toyota', 'volkswagen', 'volvo', 'mercedes']
df_lift = pd.DataFrame(index = top_10, columns = top_10)
def co_mention_counts(brand1, brand2, commentlist):
    count = 0
    for eachcomment in commentlist:
        if eachcomment.count(brand1) != 0 and eachcomment.count(brand2) != 0:
            count += 1
    return count
def mention_counts(brand, commentlist):
    count = 0
    for eachcomment in commentlist:
        if eachcomment.count(brand) != 0:
            count += 1
    return count
def lift_ratio(brand1, brand2, commentlist):
    numerator = co_mention_counts(brand1, brand2, commentlist)*len(commentlist)
    denominator = mention_counts(brand1, commentlist)*mention_counts(brand2, commentlist)
    lift = float(numerator)/denominator
    return lift
for eachbrand in top_10:
    top_10 = [brand for brand in top_10 if brand != eachbrand]
    for j in range(len(top_10)):
        df_lift[top_10[j]].ix[eachbrand] = lift_ratio(top_10[j], eachbrand, comments_list)
        df_lift[eachbrand].ix[top_10[j]] = df_lift[top_10[j]][eachbrand]
df_lift

,bmw,acura,audi,infiniti,honda,nissan,toyota,volkswagen,volvo,mercedes
bmw,NaN,1.266287,1.701216,1.567311,1.241779,0.9756976,1.422741,1.219224,1.55215,2.131125
acura,1.266287,NaN,2.18894,2.354452,2.981322,1.641705,1.412835,1.815642,2.558725,1.994681
audi,1.701216,2.18894,NaN,2.117396,1.677349,1.061819,2.059902,3.690069,3.969113,3.350002
infiniti,1.567311,2.354452,2.117396,NaN,1.640162,3.511458,2.493046,1.865386,2.700653,3.233387
honda,1.241779,2.981322,1.677349,1.640162,NaN,2.383601,2.935952,2.461525,1.992851,1.78324
nissan,0.9756976,1.641705,1.061819,3.511458,2.383601,NaN,4.06095,1.287233,1.391767,1.593293
toyota,1.422741,1.412835,2.059902,2.493046,2.935952,4.06095,NaN,2.354502,2.185708,3.46458
volkswagen,1.219224,1.815642,3.690069,1.865386,2.461525,1.287233,2.354502,NaN,3.93686,2.971592
volvo,1.55215,2.558725,3.969113,2.700653,1.992851,1.391767,2.185708,3.93686,NaN,5.533343
mercedes,2.131125,1.994681,3.350002,3.233387,1.78324,1.593293,3.46458,2.971592,5.533343,NaN


In [89]:
df_lift.to_csv("lift chart.csv")

In [166]:
#Calculate 1/lift for MDS
top_10 = ['bmw', 'acura', 'audi', 'infiniti', 'honda', 'nissan', 'toyota', 'volkswagen', 'volvo', 'mercedes']
df_dist = pd.DataFrame(index = top_10, columns = top_10)
for eachbrand in top_10:
    top_10 = [brand for brand in top_10 if brand != eachbrand]
    for j in range(len(top_10)):
        df_dist[top_10[j]].ix[eachbrand] = 1/float(lift_ratio(top_10[j], eachbrand, comments_list))
        df_dist[eachbrand].ix[top_10[j]] = df_dist[top_10[j]][eachbrand]
df_dist

,bmw,acura,audi,infiniti,honda,nissan,toyota,volkswagen,volvo,mercedes
bmw,NaN,0.7897106,0.5878147,0.6380356,0.8052965,1.024908,0.7028684,0.820194,0.6442676,0.4692358
acura,0.7897106,NaN,0.4568421,0.4247273,0.3354217,0.6091228,0.7077966,0.5507692,0.3908197,0.5013333
audi,0.5878147,0.4568421,NaN,0.4722783,0.5961789,0.94178,0.48546,0.2709977,0.2519455,0.2985073
infiniti,0.6380356,0.4247273,0.4722783,NaN,0.609696,0.284782,0.4011158,0.5360821,0.3702809,0.3092732
honda,0.8052965,0.3354217,0.5961789,0.609696,NaN,0.4195333,0.340605,0.4062522,0.5017935,0.5607771
nissan,1.024908,0.6091228,0.94178,0.284782,0.4195333,NaN,0.2462478,0.77686,0.7185111,0.6276308
toyota,0.7028684,0.7077966,0.48546,0.4011158,0.340605,0.2462478,NaN,0.4247182,0.4575176,0.2886353
volkswagen,0.820194,0.5507692,0.2709977,0.5360821,0.4062522,0.77686,0.4247182,NaN,0.2540095,0.33652
volvo,0.6442676,0.3908197,0.2519455,0.3702809,0.5017935,0.7185111,0.4575176,0.2540095,NaN,0.1807226
mercedes,0.4692358,0.5013333,0.2985073,0.3092732,0.5607771,0.6276308,0.2886353,0.33652,0.1807226,NaN


In [103]:
df_dist.to_csv('one over lift chart.csv')

In [176]:
#Top 5 attributes
#now let's use the attribute replacement sheet, and map certain words to attributes
attributes = pd.read_csv("attributes.csv")
attr = list(attributes['Replace'])
adjective = list(attributes['Search'])
#map each adjective to its respective attribute
for i in range(len(comments_list)):
    for (index, word) in enumerate(comments_list[i]):
        for (ind, eachword) in enumerate(adjective):
            if eachword == word:
                comments_list[i][index] = adjective[ind] 

In [178]:
#top 5 most frequently mentioned attributes
attribute_set = list(set(attr))
attr_counts = [(attri,whole_thing.count(attri)) for attri in attribute_set]  
df_attr = pd.DataFrame(attr_counts)
df_attr = df_attr.rename(columns={0: 'Attributes', 1: 'Frequency'}) 
df_attr = df_attr.sort_index(by='Frequency', ascending = False)
df_attr[:5]

,Attributes,Frequency
19,price,696
17,engine,611
15,performance,581
5,design,550
7,dealer,434


In [179]:
#lift to measure brand with attributes association 
top_10 = ['bmw', 'acura', 'audi', 'infiniti', 'honda', 'nissan', 'toyota', 'volkswagen', 'volvo', 'mercedes']
df_brand_attributes = pd.DataFrame(index = list(df_attr['Attributes'][:5]), columns = top_10[:5] )
for eachbrand in top_10[:5]:
    for each_attr in df_brand_attributes.index:
        df_brand_attributes[eachbrand].ix[each_attr] = lift_ratio(each_attr, eachbrand, comments_list)
df_brand_attributes

,bmw,acura,audi,infiniti,honda
price,1.742557,1.660839,1.852986,2.230218,1.657825
engine,1.616718,1.512836,1.59619,1.758382,1.61594
performance,1.605726,1.51115,1.965196,1.7887,1.528969
design,0.9363985,0.7956879,1.198598,0.8438581,1.081766
dealer,1.797147,1.586991,1.757605,1.502984,1.531366


In [206]:
#Finding aspirational associations
aspire = pd.read_csv("aspire.csv")
non_token_comments =[]
for i in range(len(comments_list)):
    string = ' '.join(comments_list[i])
    non_token_comments.append(string)
#map aspirational comments
count = 0
aspire_phrase = list(aspire['Search'])
for i in range(len(non_token_comments)):
    for eachphrase in aspire_phrase:
        if eachphrase in non_token_comments[i]:
            non_token_comments[i] = non_token_comments[i].replace(eachphrase, 'aspirational')

In [217]:
token_comments = []
for i in range(len(non_token_comments)):
    temp = non_token_comments[i].split()
    token_comments.append(temp)
top_10 = ['bmw', 'acura', 'audi', 'infiniti', 'honda', 'nissan', 'toyota', 'volkswagen', 'volvo', 'mercedes']
df_aspire = pd.DataFrame(index = [1], columns = top_10)
for eachbrand in top_10:
        df_aspire[eachbrand].ix[1] = lift_ratio('aspirational', eachbrand, token_comments)

In [218]:
df_aspire

,bmw,acura,audi,infiniti,honda,nissan,toyota,volkswagen,volvo,mercedes
1,1.645455,1.696735,1.847541,1.647602,1.810378,0.9501639,1.652128,1.343854,2.19101,1.370915
